In [42]:
import io
import os
import re
import jieba.posseg as pseg

#全局数据
IDlist = []
Agelist = []
Genderlist = []
Edulist = []
Datalist = []

Alllist = []

def mycut( strCnText ):
    words = pseg.cut(strCnText)
    nword = []
    for w in words:
        #去除一些不关心的词:[c  连词][p  介词][q  量词][t  时间词][m  量词][eng  英文字母]
        if((w.flag != 'c' and w.flag != 'p' and w.flag != 'q' and w.flag != 't'and w.flag != 'm'and w.flag != 'eng') and len(w.word)>1):
            #通过停用词库去除一些不需要考虑的词汇
            if w.word not in stopwords:
                nword.append(w.word)
            
    return nword

#开始对数据进行读取
org_cut_data_file = '/ml/sougou/orgcut.dat'
def create_org_cut_data():
    if ( os.path.exists(org_cut_data_file) == True ):
        cget = raw_input("File exist,cover it?(y / n)")
        if ( cget == 'y' ):
            os.remove( org_cut_data_file )
        else:
            return
        
    #创建文件
    print( "开始创建文件....." )
    fp = open( org_cut_data_file , 'w' )
    for i in range( 0 , nRealSize ):
    #for i in range( 0 , 10 ):
        if ( i % 100 == 0 ):
            print( "%d / %d" % ( i , nRealSize ) )
            fp.flush()
        fp.write( ','.join(mycut(Datalist[i])).encode('utf') + '\n' )
    fp.close()
        
#加载数据
def load_org_cut_data():
    if ( os.path.exists(org_cut_data_file) == False ):
        create_org_cut_data()
    lines = [line.strip().decode('utf', 'ignore') for line in open(org_cut_data_file).readlines()]
    orgcutdata = []
    for line in lines:
        orgcutdata.append( line.split(',') )
    return orgcutdata

#获得词汇词性
#获得词汇词性
def GetFlag(Text):
    try:
        Flag = SogouDict[Text]
    except:
        Flag = None

    return Flag

'''
N		名词
V		动词
ADJ		形容词
ADV		副词
CLAS	量词
ECHO	拟声词
STRU	结构助词
AUX		助词
COOR	并列连词
CONJ	连词
SUFFIX	前缀
PREFIX	后缀
PREP	介词
PRON	代词
QUES	疑问词
NUM		数词
IDIOM	成语
'''
def IsOkWord( word ):
    Flag = GetFlag( word )
    if ( Flag == None ):
        return True
    
    if ( ("ECHO" in Flag) or ("STRU" in Flag) or ("AUX" in Flag) or ("COOR" in Flag) or ("CONJ" in Flag) or ("SUFFIX" in Flag) or ("PREFIX" in Flag) ):
        return False
    if ( ("PREP" in Flag) or ("PRON" in Flag) or ("QUES" in Flag) or ("NUM" in Flag) ):
        return False
    
    return True

#对数据进行二次处理，一方面是将数据改为空格间隔，另一方面是可以对数据进行进一步的处理
def org_to_final_cut_data(org_cut_data):
    cutdata = []
    for words in org_cut_data:
        #这里可以对words进行进一步的调整    
        '''
        wordslen = len( words )
        for i in  range(0,wordslen)[::-1]:
            if ( IsOkWord( words[i].encode("utf-8") ) == False ):
                words.pop(i)
        '''     
        cutdata.append( ' '.join(words) )
        #cutdata.append( ' '.join(set(words)) )
       
    return cutdata


#-------------------------开始
#按行读取数据
file = open('/ml/sougou/user_tag_query.2W.TRAIN')
file_lines = file.readlines()
file_linenum = len(file_lines)
file.close()

#age:    0：未知年龄; 1：0-18岁; 2：19-23岁; 3：24-30岁; 4：31-40岁; 5：41-50岁; 6： 51-999岁
#gender: 0：未知1：男性2：女性
#edu:    0：未知学历; 1：博士; 2：硕士; 3：大学生; 4：高中; 5：初中; 6：小学

#通过正则表达式说去数据
pattern = '([0-9A-Z]*)\s([0-6]{1})\s([0-2]{1})\s([0-6]{1})\s(.*)'

for i in range( 0 , file_linenum ):
    m = re.search( pattern , file_lines[i] )
    if m == None:
        print( '[ERROR]bad lines:' , i )
        break
    else:
        IDlist.append(m.group(1))
        Agelist.append(m.group(2))
        Genderlist.append(m.group(3))
        Edulist.append(m.group(4))
        Datalist.append(m.group(5))
        Alllist.append( m.group(2) + m.group(3) + m.group(4) )

print( "读取原始数据%d条" % (len(IDlist)) )

#文本处理
#读取停用词
stopwords = [line.strip().decode('utf-8', 'ignore') for line in open('/ml/sougou/stopwords.txt').readlines()]
print( "读取停用词%d条" % (len(stopwords)) )

#读取搜狗词库
SogouDict = {}
file = open('/ml/sougou/SogouLabDic.dic')
SogouLabDic = file.readlines()
SogouLabDicLen = len( SogouLabDic )
pattern = '([^\s]*)\s*[0-9]*\s*(.*)'
for i in range( 0 , SogouLabDicLen ):
    m = re.search( pattern , SogouLabDic[i] )
    if m == None:
        print( '[ERROR]bad lines:' , i )
        break
    else:
        SogouDict[m.group(1).decode("gbk",'ignore').encode("utf-8")] = m.group(2)
SogouLabDicLen = len(SogouDict)
print( "读取搜狗词库%d" % SogouLabDicLen )

#处理数据
org_cut_data = load_org_cut_data()
final_cut_data = org_to_final_cut_data( org_cut_data )
print( "读取切分后数据%d条" % len(final_cut_data) )

#print( final_cut_data[0] )

#这里增加一个对性别位置数据的剔除工作
print( "原有数据个数:%d" % (len(org_cut_data)) )
for i in range( 0 , len(org_cut_data) )[::-1]:
    if (Agelist[i] == '0' or  Genderlist[i] == '0' or Edulist[i] == '0'):
    #if ( Genderlist[i] == '0' ):
        IDlist.pop(i)
        Agelist.pop(i)
        Genderlist.pop(i)
        Edulist.pop(i)
        Alllist.pop(i)
        final_cut_data.pop(i)
        org_cut_data.pop(i)

print ( "剔除未知之后数据个数:%d" % (len(org_cut_data)) )


读取原始数据20000条
读取停用词2612条
读取搜狗词库157202
读取切分后数据20000条
原有数据个数:20000
剔除未知之后数据个数:17663


In [43]:
#切分数据

import scipy as sp  
import numpy as np  
from sklearn.datasets import load_files  
from sklearn.cross_validation import train_test_split  
from sklearn.feature_extraction.text import  TfidfVectorizer
  
doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(final_cut_data, Genderlist, test_size = 0.3)  
#doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(final_cut_data, Agelist, test_size = 0.3)
#doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(final_cut_data, Edulist, test_size = 0.3)
#doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(final_cut_data, Alllist, test_size = 0.3)
#doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(org_cut_data, Genderlist, test_size = 0.3)  
print( "train:%d , test:%d" % ( len(doc_terms_train) , len(doc_terms_test) ) )

train:12364 , test:5299


In [56]:
#转换
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


#********FOR None
vectorizer = TfidfVectorizer( max_features = 5000 )
print( vectorizer )
'''
#********FOR LogisticRegression
vectorizerL = TfidfVectorizer(max_df=0.5 , max_features = None , ngram_range = (1, 2) , norm = 'l2' ,use_idf =True )

#********FOR MultinomialNB
vectorizerM = TfidfVectorizer(max_df = 0.5 , max_features = 10000 , ngram_range = (1, 2) , norm = 'l2')

#********FOR BernoulliNB
vectorizerB = TfidfVectorizer(max_df=0.5 , max_features=10000 , ngram_range=(1, 1) , norm='l1', use_idf=True)
'''
'''
x_train = vectorizer.fit_transform(doc_terms_train)
x_test = vectorizer.transform(doc_terms_test)

x_trainL = vectorizerL.fit_transform(doc_terms_train)
x_testL = vectorizerL.transform(doc_terms_test)

x_trainM = vectorizerM.fit_transform(doc_terms_train)
x_testM = vectorizerM.transform(doc_terms_test)

x_trainB = vectorizerB.fit_transform(doc_terms_train)
x_testB = vectorizerB.transform(doc_terms_test)
'''

#vectorizer = CountVectorizer()

#x_vtrain = vectorizer.fit_transform(doc_terms_train)
#x_vtest = vectorizer.transform(doc_terms_test)

#print( vectorizer )

print( "ok" )


TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
ok


有待考虑：TfidfVectorizer ？= CountVectorizer + TfidfTransformer
'''python
#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
vectorizer=CountVectorizer()

#该类会统计每个词语的tf-idf权值
transformer=TfidfTransformer()

x_vtrain = vectorizer.fit_transform(doc_terms_train)
x_vtest = vectorizer.transform(doc_terms_test)

#第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵
x_train = transformer.fit_transform(x_vtrain)
x_test = transformer.transform(x_vtest)

#x_train = vectorizer.fit_transform(doc_terms_train)
#x_test = vectorizer.transform(doc_terms_test)

allwords = vectorizer.get_feature_names()
print( "当前所有词汇数量%d" % ( len(allwords) ) )
'''

In [53]:
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
"""
model = GradientBoostingClassifier(n_estimators=200)  
    model.fit(train_x, train_y)  
    return model  
"""
#0.802315681934
def fun_LogisticRegression( x_train , y_train , x_test , y_test ):
    from sklearn.linear_model.logistic import LogisticRegression
    #model = LogisticRegression()
    model = LogisticRegression(C=2,penalty='l2')
    clf = model.fit(x_train , y_train )
    predictions = clf.predict( x_test )
    print( precision_score(y_test, predictions, average='micro')   )
    
#0.801464328282
def fun_MultinomialNB( x_train , y_train , x_test , y_test ):
    from sklearn.naive_bayes import MultinomialNB  
    model = MultinomialNB(alpha = 0.01)
    clf = model.fit(x_train , y_train )
    predictions = clf.predict( x_test )
    print( precision_score(y_test, predictions, average='micro')   )

#0.811510301379
def fun_BernoulliNB( x_train , y_train , x_test , y_test ):
    from sklearn.naive_bayes import BernoulliNB  
    model = BernoulliNB(alpha=0.01)
    clf = model.fit(x_train , y_train )
    predictions = clf.predict( x_test )
    print( precision_score(y_test, predictions, average='micro')   )
    
def fun_KNeighborsClassifier( x_train , y_train , x_test , y_test ):
    from sklearn.neighbors import KNeighborsClassifier
    model = KNeighborsClassifier()
    clf = model.fit(x_train , y_train )
    predictions = clf.predict( x_test )
    print( precision_score(y_test, predictions, average='micro')   )
    
def fun_DecisionTreeClassifier( x_train , y_train , x_test , y_test ):
    from sklearn.tree import DecisionTreeClassifier
    model = DecisionTreeClassifier()
    clf = model.fit(x_train , y_train )
    predictions = clf.predict( x_test )
    print( precision_score(y_test, predictions, average='micro')   )
    
def fun_SVC( x_train , y_train , x_test , y_test ):
    from sklearn.svm import SVC
    model = SVC()
    clf = model.fit(x_train , y_train )
    predictions = clf.predict( x_test )
    print( precision_score(y_test, predictions, average='micro')   )
    

In [54]:

print( "No param:" )
fun_LogisticRegression( x_train , y_train , x_test , y_test )
fun_MultinomialNB( x_train , y_train , x_test , y_test )
fun_BernoulliNB( x_train , y_train , x_test , y_test )
'''
print( "TfidfVectorizer For LogisticRegression:" )
fun_LogisticRegression( x_trainL , y_train , x_testL , y_test )
fun_MultinomialNB( x_trainL , y_train , x_testL , y_test )
fun_BernoulliNB( x_trainL , y_train , x_testL , y_test )

print( "TfidfVectorizer For MultinomialNB:" )
fun_LogisticRegression( x_trainM , y_train , x_testM , y_test )
fun_MultinomialNB( x_trainM , y_train , x_testM , y_test )
fun_BernoulliNB( x_trainM , y_train , x_testM , y_test )

print( "TfidfVectorizer For BernoulliNB:" )
fun_LogisticRegression( x_trainB , y_train , x_testB , y_test )
fun_MultinomialNB( x_trainB , y_train , x_testB , y_test )
fun_BernoulliNB( x_trainB , y_train , x_testB , y_test )
'''

#fun_SVC( x_train , y_train , x_test , y_test )
#fun_DecisionTreeClassifier( x_train , y_train , x_test , y_test )
#fun_KNeighborsClassifier( x_train , y_train , x_test , y_test )

No param:
0.797697678807
0.805246272882
0.812983581808


'\nprint( "TfidfVectorizer For LogisticRegression:" )\nfun_LogisticRegression( x_trainL , y_train , x_testL , y_test )\nfun_MultinomialNB( x_trainL , y_train , x_testL , y_test )\nfun_BernoulliNB( x_trainL , y_train , x_testL , y_test )\n\nprint( "TfidfVectorizer For MultinomialNB:" )\nfun_LogisticRegression( x_trainM , y_train , x_testM , y_test )\nfun_MultinomialNB( x_trainM , y_train , x_testM , y_test )\nfun_BernoulliNB( x_trainM , y_train , x_testM , y_test )\n\nprint( "TfidfVectorizer For BernoulliNB:" )\nfun_LogisticRegression( x_trainB , y_train , x_testB , y_test )\nfun_MultinomialNB( x_trainB , y_train , x_testB , y_test )\nfun_BernoulliNB( x_trainB , y_train , x_testB , y_test )\n'

In [57]:
from sklearn.grid_search import GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model.logistic import LogisticRegression

pipeline = Pipeline([
#    ('vect', TfidfVectorizer(max_features=10000,ngram_range=(1, 1), max_df=0.5, norm='l1', use_idf=True)),
#    ('clf', BernoulliNB(alpha=0.01)),
    
#    ('vect', TfidfVectorizer(max_df = 0.5 , max_features = 10000 , ngram_range = (1, 2) , norm = 'l2')),
#    ('clf', MultinomialNB(alpha = 0.01))
    
    ('vect', TfidfVectorizer()),
    ('clf', LogisticRegression(C=2,penalty='l2'))
])
parameters = {

    #'vect__max_df': (0.25, 0.5, 0.75),
    #'vect__stop_words': ('english', None ),
    'vect__max_features': (  10000,15000, None ),
    #'vect__ngram_range': ((1, 1), (1, 2)),
    #'vect__use_idf': ( True , False ),
    #'vect__norm': ('l1', 'l2'),
    
    #'clf__penalty': ('l1', 'l2'),
    #'clf__C': (0.01, 0.1, 1, 2 , 3)
    #'clf__alpha': ( 0.1, 0.01, 0.001, 0.0001)  
}


grid = GridSearchCV( pipeline , parameters, n_jobs=-1, verbose=1, scoring='accuracy', cv=4)
grid.fit(doc_terms_train, y_train)
#grid.fit(x_train, y_train)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
best_parameters = grid.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print(' \t %s: %r' % (param_name, best_parameters[param_name]))

#predictions = grid_search.predict(X_test)


Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   55.8s finished


GridSearchCV(cv=4, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'vect__max_features': (10000, 15000, None)},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=1)
0.804674862504
 	 vect__max_features: None
